# CODE

In [2]:
# ============================================================
# @title INSTALL – Agnostic Agent (GitHub) + Infra Qwen3
# ============================================================
import os, sys, subprocess, importlib

def sh(cmd: str):
    return subprocess.run(cmd, shell=True, text=True, capture_output=True)

def install_if_missing(package_name, import_name=None):
    import_name = import_name or package_name.split('>=')[0].split('==')[0].replace('-', '_')
    try: 
        importlib.import_module(import_name)
    except ImportError:
        print(f"📦 Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])

# 1. SUPER CLEANUP & UPDATE
if os.path.exists("repo_agnostic"):
    print("🔄 Repo exists. Pulling latest changes...")
    os.system("cd repo_agnostic && git pull")
else:
    print("🧹 Cleaning up shadowing folders...")
    os.system("rm -rf agnostic_agent repo_agnostic")
    print("📥 Cloning repo...")
    os.system("git clone https://github.com/JacoboGGLeon/agnostic_agent.git repo_agnostic")

# Always reinstall editable to be sure (fast/idempotent usually)
print("🛠 Updating package install...")
os.system("pip install -q -e repo_agnostic")

# 2. FIX PATH
pkg_path = os.path.abspath("repo_agnostic")
if pkg_path not in sys.path: sys.path.insert(0, pkg_path)

# 3. OTHER DEPENDENCIES (Idempotent)
deps = [
    ("vllm>=0.9.0", "vllm"),
    ("huggingface_hub>=0.23.0", "huggingface_hub"),
    ("openai==1.108.2", "openai"),
    ("langchain", "langchain"),
    ("langchain-core", "langchain_core"),
    ("langchain-openai", "langchain_openai"),
    ("langgraph", "langgraph"),
    ("pydantic>=2.7.0", "pydantic"),
    ("pyyaml>=6.0", "yaml"),
    ("streamlit", "streamlit"),
    ("sqlite-vec", "sqlite_vec"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
]

for pkg, imp in deps:
    install_if_missing(pkg, imp)

try:
    importlib.import_module("langchain_qwq_vllm")
except ImportError:
    print("📦 Installing langchain-qwq-vllm...")
    os.system("pip install -q git+https://github.com/whynpc9/langchain-qwq-vllm.git")

print("✅ Agnostic Agent INSTALLED and ready!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━

In [4]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Dec  9 14:41 .
drwxr-xr-x 1 root root 4096 Feb  6 19:38 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


# DEFINE vLLM

In [ ]:
# TRY

#@title DOWNLOAD MODELS – LLM, Embedding, Reranker (Qwen3)
from agnostic_agent import prepare_qwen_models

# IDs de modelos (puedes ajustarlos)
LLM_MODEL_ID = "Qwen/Qwen3-0.6B"  #@param ["Qwen/Qwen3-0.6B", "Qwen/Qwen3-0.6B-Base", "Qwen/Qwen3-4B", "Qwen/Qwen3-4B-Instruct-2507", "Qwen/Qwen3-4B-Thinking-2507"]

EMB_MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"  #@param ["Qwen/Qwen3-Embedding-0.6B"]

RERANK_MODEL_ID = "Qwen/Qwen3-Reranker-0.6B"  #@param ["Qwen/Qwen3-Reranker-0.6B"]

print("⬇️ Descargando / preparando modelos Qwen3 (prepare_qwen_models)...")
model_paths = prepare_qwen_models(
    llm_model_id=LLM_MODEL_ID,
    emb_model_id=EMB_MODEL_ID,
    rerank_model_id=RERANK_MODEL_ID,
    base_dir="LM_MODEL",
)
print("\n✅ Modelos listos en disco:")
print("  LLM   :", model_paths.llm_dir)
print("  EMB   :", model_paths.emb_dir)
print("  RERANK:", model_paths.rerank_dir)

import os
os.environ["LLM_MODEL_ID"]   = LLM_MODEL_ID
os.environ["EMB_MODEL_ID"]   = EMB_MODEL_ID
os.environ["RERANK_MODEL_ID"] = RERANK_MODEL_ID


In [ ]:
#@title LIMPIEZA LIGERA DE GPU (opcional)
import gc
try:
    import torch
    has_torch = True
except ImportError:
    has_torch = False

print("🧹 Limpiando referencias de Python...")
gc.collect()
if has_torch and torch.cuda.is_available():
    print("🧠 Vaciando caché CUDA...")
    torch.cuda.empty_cache()
else:
    print("ℹ️ Torch CUDA no disponible o sin GPU visible.")


In [ ]:
#@title DEPLOY vLLM SERVER – SOLO LLM (chat) · L4/T4-friendly
import transformers
from agnostic_agent import VllmConfig, start_qwen_vllm_servers

print("Transformers version:", transformers.__version__)

VLLM_HOST = os.getenv("VLLM_HOST", "127.0.0.1")
LLM_PORT = 8000
EMB_PORT = 8001
RERANK_PORT = 8002
VLLM_MODE = "POWER"  # "FAST", "MEDIUM", "POWER", "LIMIT"

print(f"\n🚀 Lanzando servidor vLLM (start_qwen_vllm_servers)...")
print(f"🔧 VLLM_MODE = {VLLM_MODE}")

if VLLM_MODE == "FAST":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.40,
        llm_max_len=2048,
        llm_max_num_seqs=4,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "MEDIUM":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.55,
        llm_max_len=4096,
        llm_max_num_seqs=6,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "POWER":
    # Modo POWER “cargado” para L4 + Qwen3-0.6B:
    # - Más contexto (16k tokens)
    # - Menos concurrencia (4 secuencias)
    # - GPU util ~0.8 para exprimir la L4 sin matarla
    vllm_cfg = VllmConfig(
        # config base
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        # language model
        llm_gpu_util=0.90,   # antes 0.70
        llm_max_len=32768,   # antes 8192  ← clave
        llm_max_num_seqs=1,  # antes 8     ← menos batch, más contexto
        # embedding model
        emb_gpu_util=0.05,
        emb_max_len=1024,
        emb_max_num_seqs=1,
        # reranker model
        rerank_gpu_util=0.05,
        rerank_max_len=1024,
        rerank_max_num_seqs=1,
        # config extra
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )

elif "LIMIT":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.85,    # un poco más agresivo
        llm_max_len=32768,    # 🔥 32k tokens de contexto
        llm_max_num_seqs=2,   # menos batch, más contexto por secuencia
        emb_gpu_util=0.35,
        emb_max_len=1024,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.35,
        rerank_max_len=1024,
        rerank_max_num_seqs=4,
        enable_reasoning=True, # True | False
        tool_call_parser="hermes",
        reasoning_parser="qwen3",    # "qwen3" | none
        start_emb_server=True,
        start_rerank_server=False,
    )

endpoints, servers = start_qwen_vllm_servers(
    model_paths=model_paths,
    config=vllm_cfg,
    set_env=True,
)

print("\n✅ Servidor vLLM (LLM) listo.")


# RUN EXPERIMENT

In [ ]:
#@title streamlit_app.py (From Git)
import os, shutil

if os.path.exists("repo_agnostic/streamlit_app.py"):
    shutil.copy("repo_agnostic/streamlit_app.py", ".")
    print("✅ streamlit_app.py copied from repo_agnostic")
else:
    print("❌ ERROR: streamlit_app.py not found in repo_agnostic")


In [ ]:
#@title .streamlit/config.toml (From Git)
import os, shutil

!mkdir -p .streamlit
if os.path.exists("repo_agnostic/.streamlit/config.toml"):
    shutil.copy("repo_agnostic/.streamlit/config.toml", ".streamlit/config.toml")
    print("✅ .streamlit/config.toml copied from repo_agnostic")
else:
    print("❌ ERROR: .streamlit/config.toml not found in repo_agnostic")


In [ ]:
#@title .streamlit/config.toml (From Git)
import os, shutil

!mkdir -p .streamlit
if os.path.exists("repo_agnostic/.streamlit/config.toml"):
    shutil.copy("repo_agnostic/.streamlit/config.toml", ".streamlit/config.toml")
    print("✅ .streamlit/config.toml copied from repo_agnostic")
else:
    print("❌ ERROR: .streamlit/config.toml not found in repo_agnostic")


In [ ]:
#@title Lanzar servicio Streamlit en Colab (window o iframe)
import subprocess, time, os

# ---- PARAMS (Colab UI) ----
PORT = 8501  #@param {type:"integer"}
MODE = "iframe"  #@param ["window", "iframe"]
IFRAME_HEIGHT = 800  #@param {type:"integer"}
IFRAME_WIDTH = "100%"  #@param {type:"string"}

# ---- Colab output (solo si existe) ----
try:
    from google.colab import output
    _IN_COLAB = True
except Exception:
    output = None
    _IN_COLAB = False

# ---- Mata procesos previos (opcional) ----
os.system('pkill -f "streamlit run streamlit_app.py" || true')
os.system('pkill -f "streamlit" || true')

# ---- Lanza Streamlit ----
cmd = [
    "streamlit", "run", "streamlit_app.py",
    "--server.address", "0.0.0.0",
    "--server.port", str(PORT),
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
]

streamlit_process = subprocess.Popen(cmd)
time.sleep(3)

# ---- Exponer ----
if _IN_COLAB and output is not None:
    if MODE == "iframe":
        output.serve_kernel_port_as_iframe(
            PORT,
            width=IFRAME_WIDTH,
            height=IFRAME_HEIGHT,
        )
    else:
        output.serve_kernel_port_as_window(
            PORT,
            anchor_text="🔗 Abrir app Streamlit en una nueva pestaña"
        )
else:
    print(f"Streamlit corriendo en http://127.0.0.1:{PORT} (no-Colab).")
